In [4]:
%logstop
%logstart -rtq ~/.logs/pw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [5]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

mkdir: cannot create directory ‘pw-data’: File exists
File ‘./pw-data/201701scripts_sample.json.gz’ already there; not retrieving.

File ‘./pw-data/practices.json.gz’ already there; not retrieving.



## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [6]:
import gzip
import simplejson as json

In [7]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [5]:
scripts[:2]

[{'bnf_code': '0101010G0AAABAB',
  'items': 2,
  'practice': 'N81013',
  'bnf_name': 'Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
  'nic': 5.98,
  'act_cost': 5.56,
  'quantity': 1000},
 {'bnf_code': '0101021B0AAAHAH',
  'items': 1,
  'practice': 'N81013',
  'bnf_name': 'Alginate_Raft-Forming Oral Susp S/F',
  'nic': 1.95,
  'act_cost': 1.82,
  'quantity': 500}]

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [7]:
practices[:2]

[{'code': 'A81001',
  'name': 'THE DENSHAM SURGERY',
  'addr_1': 'THE HEALTH CENTRE',
  'addr_2': 'LAWSON STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 1HU'},
 {'code': 'A81002',
  'name': 'QUEENS PARK MEDICAL CENTRE',
  'addr_1': 'QUEENS PARK MEDICAL CTR',
  'addr_2': 'FARRER STREET',
  'borough': 'STOCKTON ON TEES',
  'village': 'CLEVELAND',
  'post_code': 'TS18 2AW'}]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [8]:

def describe(key):
    
    try: 
        total = sum (scripts[i][key] for i in range(len(scripts)))
        avg = total/len(scripts)
        s =  (sum((float(scripts[i][key] - avg)**2 for i in range(0,len(scripts))))/(len(scripts) - 1))**0.5
        sorted_key = sorted(scripts, key = lambda i: i[key])
        #sorted_key = scripts[i][key].sort()
        q25 = sorted_key[int(len(scripts)/4)][key]
        med = sorted_key[int(len(scripts)/2)][key]
        q75 = sorted_key[int(len(scripts)*3/4)][key]
    except ValueError: 
        print("oops")
       
        
    return (total, avg, s, q25, med, q75)

In [9]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]
print(summary)

[('items', (4410054, 11.522744731217633, 33.11220959820685, 1, 3, 8)), ('quantity', (316356836, 826.5883059943667, 3872.186073305146, 30, 120, 466)), ('nic', (29048309.790000338, 75.89844899484315, 197.57308474088356, 7.7, 22.62, 65.94)), ('act_cost', (27053937.599999707, 70.68748295124895, 183.26755837716715, 7.25, 21.24, 61.53))]


In [10]:
grader.score.pw__summary_statistics(summary)

Your score:  1.0


## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

In [11]:
bnf_names = [scripts[key]['bnf_name'] for key in range(len(scripts))]

In [12]:
scripts[1]['bnf_name']
bnf_names[0:5]

['Co-Magaldrox_Susp 195mg/220mg/5ml S/F',
 'Alginate_Raft-Forming Oral Susp S/F',
 'Sod Algin/Pot Bicarb_Susp S/F',
 'Sod Alginate/Pot Bicarb_Tab Chble 500mg',
 'Gaviscon Infant_Sach 2g (Dual Pack) S/F']

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [13]:
groups = {name: [] for name in bnf_names}
for script in scripts:
    groups[script['bnf_name']].append(script) 

In [14]:
#for script in scripts:
#   print(script['bnf_name'])

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [15]:
max_item = []
item_values = []
item_total = 0
tuple_list = []

for k, v in groups.items():
    item_name = k
    item_total = 0
    item_values = v
    for value in item_values:
        item_total = item_total + value['items']
        
    tuple_list.append(tuple([item_name, item_total]))
    
tuple_list.sort(key = lambda x: x[1]) 
max_item.append(tuple_list[-1])
max_item
    
        

[('Omeprazole_Cap E/C 20mg', 113826)]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [16]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [28]:
def group_by_field(data, fields):
    new_dict = {} 
    groupby_titles = dict(fields)

# groups = {name: [] for name in bnf_names} or script in scripts:
# groups[script['bnf_name']].append(script) 



https://www.saltycrane.com/blog/2014/10/example-using-groupby-and-defaultdict-do-same-task/

In [ ]:
groups = group_by_field(scripts, ('bnf_name',))
test_max_item = max(group['item'] for group in groups)

assert test_max_item == max_item

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [8]:
from collections import defaultdict
group_by = defaultdict(list)
for practice in practices:
    #for key,value in practice.items():
        key_string = practice['code']
        mylist = []
        mylist.append(practice)
        group_by[key_string].append(mylist)
    

    
practice_postal = dict(group_by)
for code in practice_postal.keys():
        for i in range(len(practice_postal[code])):
            if practice_postal[code][0][0]['post_code'] == min((str(practice_postal[code][0][0]['post_code'])), str(practice_postal[code][i][0]['post_code'])):
                None
            else:
                practice_postal[code][0][0].update(practice_postal[code][i][0])

for key,value in practice_postal.items():
    if key == '0':
        print("there is  0 value")
    else: 
        practice_postal[key] = value[0][0]['post_code']  
        
practice_postal['K82019']
        


'HP21 8TR'

https://stackoverflow.com/questions/13264511/typeerror-unhashable-type-dict

#for key,value in practice_postal.items():
  if key == '0':
       print("there is  0 value")
    else: 
    #    practice_postal[key] = value[0]   
#practice_postal['K82019']
#group_by['K82019']  
test_pref = 'K82019'
#for practice in practices:
res = {key:val for key, val in group_by.items() if key.startswith(test_pref)} 
  
print("Filtered dictionary keys are : " + str(res)) 


test_pref = 'K82019'
for practice in practices:
    for k,v in practice.items():
        if practice[k] == 'K82019':
            print("------------------------")
            print(practice)

    

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [9]:
assert practice_postal['K82019'] == 'HP21 8TR'


Now we can join `practice_postal` to `scripts`.

In [10]:
joined = scripts[:]
for abc in joined:
    abc['post_code'] = practice_postal[abc['practice']]
     

In [11]:
postjoinedcode = [joined[i]['post_code'] for i in range(len(joined))]
newgroups = {name: [] for name in postjoinedcode}
for abc in joined:
    newgroups[abc['post_code']].append(abc) 

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [12]:

item_values = []

tuple_list = []

for k, v in newgroups.items():
    item_total = 0
    postcode = k
    item_values = v
    for value in item_values:
        item_total = item_total + value['items']
        
    tuple_list.append(tuple([postcode, item_total]))
    
tuple_list.sort(key = lambda x: x[0]) 

my_100_list = tuple_list[:100]
           
            


In [25]:
postal_totals = my_100_list

grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [13]:
from collections import defaultdict
total_items_by_bnf_post = defaultdict(list)

for script in joined:
    total_items_by_bnf_post[script['post_code'], script['bnf_name']].append(script)
   
sorted_dict = dict(sorted(total_items_by_bnf_post.items()))


In [14]:
total_items_by_bnf_post = sorted_dict

    
    
assert len(total_items_by_bnf_post) == 141196

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [15]:

total_items = []
newdict = defaultdict(list)
for key,value in total_items_by_bnf_post.items():
    newdict[key[0]].append(value)
    

#print("here is my new dict")
#newdict
        
for key, value in newdict.items(): 
    l = {}
    l[key] = value
    total_items.append(l)
    

    
    
        

    
total_items_by_post = total_items    
len(total_items)
    
#assert len(total_items_by_post) == 118


118

In [15]:
assert len(total_items_by_post) == 118

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [16]:
max_item_by_post = []

puple_list = []
count = 0
for onedef_dict in total_items:
        for k,v in onedef_dict.items():
            item_total = 0
            postcode= k
            for value in v:
                item_total = 0
                #print("the length of this v[1] is " + str(len(v[1])))
                #print(value)
                #print("------------------------") 
                
                for val in value:
                    
                    item_total = item_total + int(val['items'])
                    bnf_name = val['bnf_name'] 
                    #print("the bnf name assigned is " + val['bnf_name'] )
                l = tuple([postcode, bnf_name, item_total])
                    #print (l)
                puple_list.append(l)
        #print("---------Every POstal change--------------------------- ")
        
puple_list[:10]  
#print(len(puple_list))
#print(count)
#print(len)



[('B11 4BW', '3m Health Care_Cavilon Durable Barrier C', 7),
 ('B11 4BW', '3m Health Care_Cavilon No Sting Barrier', 2),
 ('B11 4BW', 'Abidec_Dps', 63),
 ('B11 4BW', 'Acetic Acid_Ear Spy 2% 5ml', 9),
 ('B11 4BW', 'Aciclovir_Crm 5%', 13),
 ('B11 4BW', 'Aciclovir_Tab 200mg', 4),
 ('B11 4BW', 'Aciclovir_Tab 800mg', 9),
 ('B11 4BW', 'Aciclovir_Tab Disper 400mg', 2),
 ('B11 4BW', 'Adalat LA 30_Tab 30mg', 3),
 ('B11 4BW', 'Adalat LA 60_Tab 60mg', 11)]

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [47]:
from operator import itemgetter, attrgetter

s = sorted(puple_list, key=itemgetter(2), reverse=True)

finaltup = sorted(s, key=itemgetter(0))
prevtup = finaltup[0][0]
postsum = 0
r = 0
postalsumlist = []
for tup in finaltup:
    if str(tup[0]) == str(prevtup):
        
        postsum = postsum + tup[2]
        #print('jndvc')
        prevtup = tup[0]
        r+=1
        if r == (len(finaltup)-1):
            postalsumlist.append(postsum) 
            break
    else:
        r +=1
        postalsumlist.append(postsum)
        postsum = tup[2]
        prevtup = tup[0]
        #print('jndvc')
        None

i=0 
prevtup = finaltup[0][0]
tupandtotallist = []
for tup in finaltup:
    if str(tup[0]) == str(prevtup):
        tupandtotallist.append(postalsumlist[i])
        
    else:
        prevtup = tup[0]
        i +=1
        tupandtotallist.append(postalsumlist[i])
      
print(postalsumlist[0:10])
i= 0
ratiotup = []
prevtup = finaltup[0]
for tup in finaltup:
    n = tup[2]/tupandtotallist[i]
    i +=1
    ratiotup.append(tuple([tup[0], tup[1],  n]))
    

counter = 0
postlist = []
firstpost100 = []
for tup in ratiotup:
    if tup[0] not in postlist and counter <100:
        firstpost100.append(tup)
        postlist.append(tup[0])
        counter +=1
        
    else: 
        None

firstpost100         

[20673, 19001, 29103, 24859, 36531, 34356, 28254, 54514, 29388, 44585]


[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.03415082474725487),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.02926161780958897),
 ('B21 9RY', 'Metformin HCl_Tab 500mg', 0.03549462254750369),
 ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.024095900880968663),
 ('B70 7AW', 'Paracet_Tab 500mg', 0.0266896608360023),
 ('BB11 2DL', 'Omeprazole_Cap E/C 20mg', 0.02884503434625684),
 ('BB2 1AX', 'Omeprazole_Cap E/C 20mg', 0.03645501521908402),
 ('BB3 1PY', 'Omeprazole_Cap E/C 20mg', 0.03428477088454342),
 ('BB4 5SL', 'Omeprazole_Cap E/C 20mg', 0.040696883081529876),
 ('BB7 2JG', 'Omeprazole_Cap E/C 20mg', 0.029471795446899183),
 ('BB8 0JZ', 'Atorvastatin_Tab 20mg', 0.022563442442074293),
 ('BB9 7SR', 'Omeprazole_Cap E/C 20mg', 0.023833193804939305),
 ('BD3 8QH', 'Atorvastatin_Tab 40mg', 0.03422179914326511),
 ('BH18 8EE', 'Omeprazole_Cap E/C 20mg', 0.029000583563798747),
 ('BH23 3AF', 'Omeprazole_Cap E/C 20mg', 0.03733292364418497),
 ('BL1 8TU', 'Omeprazole_Cap E/C 20mg', 0

In [48]:
items_by_region = firstpost100[:100]

In [49]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*